# LA County COVID-19 Cases by Neighborhoods

Use LA County's [countywide statistical areas](http://geohub.lacity.org/datasets/lacounty::countywide-statistical-areas-csa) and look at trends in cases and new cases by neighborhood.

**The 25th percentile and 75th percentile of cases are calculated for each day and drawn in gray. The blue line is the neighborhood's 7-day rolling average of cases or new cases.** The percentiles provide context to whether a neighborhood's cases are relatively high or lower compared to other parts of the county.

In [4]:
import altair as alt
import ipywidgets
import pandas as pd

import default_parameters 
S3_FILE_PATH = "s3://public-health-dashboard/jhu_covid19/"

#---------------------------------------------------------------#
# Chart parameters
#---------------------------------------------------------------#
navy = "#1696D2"
gray = "#515252"

# These colors are used for the shading on cases/deaths
light_gray = "#EAEBEB"

title_font_size = 10
font_name = "Arial"
grid_opacity = 0.4
domain_opacity = 0.4
stroke_opacity = 0
time_unit = "monthdate"
chart_width = 250
chart_height = 200
bin_spacing = 100
fulldate_format = "%-m/%-d/%y"
monthdate_format = "%-m/%-d"

two_weeks_ago = default_parameters.two_weeks_ago

In [5]:
def make_chart(df, neighborhood, chart_type):
    if chart_type == "cases":
        plot_col = "cases_avg7"
        chart_title = f"{neighborhood}: Cases"
    if chart_type == "new_cases":
        plot_col = "new_cases_avg7"
        chart_title = f"{neighborhood}: New Cases"

    # Make cases charts
    cases_line = (
        alt.Chart(df.drop(columns = "date"))
        .mark_line()
        .encode(
            x=alt.X("date2", timeUnit=time_unit, 
                    title="date", axis=alt.Axis(format=monthdate_format)
                   ),
            y=alt.Y(plot_col, title="7-day avg"),
            color=alt.value(navy),
        )
    )
    
    cases_shaded = (
        alt.Chart(df[df.date >= two_weeks_ago].drop(columns = "date"))
        .mark_area()
        .encode(
            x=alt.X("date2", timeUnit = time_unit,
                   title = "date", axis=alt.Axis(format=monthdate_format)
                   ),
            y=alt.Y(plot_col, title="7-day avg"),
            color=alt.value(light_gray)
        )
    )
    
    ptile25_line = (
        alt.Chart(df.drop(columns = "date"))
        .mark_line()
        .encode(
            x=alt.X("date2", timeUnit=time_unit, 
                    title="date", axis=alt.Axis(format=monthdate_format)
                   ),
            y=alt.Y("ptile25", title="7-day avg"),
            color=alt.value(gray),
        )
    )
    
    ptile75_line = (
        alt.Chart(df.drop(columns = "date"))
        .mark_line()
        .encode(
            x=alt.X("date2", timeUnit=time_unit, 
                    title="date", axis=alt.Axis(format=monthdate_format)
                   ),
            y=alt.Y("ptile75", title="7-day avg"),
            color=alt.value(gray),
        )
    )
    
    if chart_type == "cases":
        chart = (
            (cases_shaded + cases_line + ptile25_line + ptile75_line)
              .properties(
                  title=chart_title, width=chart_width, height=chart_height
                ).configure_title(
                fontSize=title_font_size, font=font_name, anchor="middle", color="black"
            )
            .configure_axis(gridOpacity=grid_opacity, domainOpacity=domain_opacity)
            .configure_view(strokeOpacity=stroke_opacity)
            )   
            
    
    if chart_type == "new_cases":
        chart = (
            (cases_shaded + cases_line)
            .properties(
                  title=chart_title, width=chart_width, height=chart_height
                )
            .configure_title(
                fontSize=title_font_size, font=font_name, anchor="middle", color="black"
            )
            .configure_axis(gridOpacity=grid_opacity, domainOpacity=domain_opacity)
            .configure_view(strokeOpacity=stroke_opacity)
            )   
    
    return chart

In [6]:
df = pd.read_parquet(f"{S3_FILE_PATH}lacounty-neighborhood-time-series.parquet")


# There are some duplicates!
# Add the case rate (per 100k) as an additional chart

df = df.sort_values(["Region", "date"]).reset_index(drop=True)


neigborhood_dropdown = ipywidgets.Dropdown(
    description="Neighborhood",
    options=df.Region.unique()
)

output = ipywidgets.Output()

display(neigborhood_dropdown)
display(output)


def on_neighborhood_selection(*args):
    output.clear_output(wait=True)
    neighborhood = neigborhood_dropdown.value
    
    subset_df = df[df.Region == neighborhood]
    
    with output:
        display(make_chart(subset_df, neighborhood, "cases"))
        display(make_chart(subset_df, neighborhood, "new_cases"))

on_neighborhood_selection()
neigborhood_dropdown.observe(on_neighborhood_selection, names="value")

Dropdown(description='Neighborhood', options=('Acton', 'Adams', 'Agoura Hills', 'Agua Dulce', 'Alhambra', 'Als…

Output()